In [1]:
import requests
html = requests.get('https://afisha.tut.by/day/film/2020-11-01/2020-11-28/')

In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html.content, 'html.parser')

In [187]:
films = []
for film in soup.find_all('li',class_='lists__li'):
    a_name = film.find('a', class_='name')
    img = film.find('img')

    if not a_name or "news" in a_name['href']:
        continue
   
    full_film_soup = BeautifulSoup(requests.get(a_name['href']).content, 'html.parser')
    
    if full_film_soup.find("td", class_="genre"):
        genres = [genre for genre in full_film_soup.find("td", class_="genre").get_text(separator=' ').split(' ')]
        
    if full_film_soup.find("td", class_="year"):
        year = full_film_soup.find("td", class_="year").text
        
    if full_film_soup.find("td", class_="duration"):
        duration = full_film_soup.find("td", class_="duration").text
        
    if full_film_soup.find("div", class_="rating-big"):
        rating = full_film_soup.find("div", class_="rating-big").text.strip()
        
    producer = full_film_soup.find(lambda tag:tag.name=="p" and "Режиссер" in tag.text)
    film_description = full_film_soup.find("div", class_="post").h2.next_element.next_element.strip()

    actors_html = full_film_soup.find(lambda tag:tag.name=="p" and "В ролях:" in tag.text)
    if actors_html:
        actors = [x.strip() for x in actors_html.text[8:].split(',')]
    
    films.append({
        'name': a_name.text.strip(),
        'img_url': img['src'],
        'url': a_name['href'],
        'description': film_description,
        'genres': genres,
        'year': year,
        'duration': duration,
        'producer': producer,
        'actors': actors,
        'rating': rating
    })


In [188]:
import pandas as pd

df_films = pd.DataFrame(films)

In [189]:
df_films

,name,img_url,url,description,genres,year,duration,producer,actors,rating
0,Подольские курсанты,https://img.afisha.tut.by/static/media/280x300...,https://afisha.tut.by/film/podolskie-kursanty/...,Фильм о подвиге подольских курсантов в октябре...,"[Военный, Драма, Исторический]",2019,132 минуты,[Режиссер: Вадим Шмелев],"[Алексей Бардуков, Евгений Дятлов, Сергей Безр...","7,9"
1,Кольская сверхглубокая,https://img.afisha.tut.by/static/media/280x300...,https://afisha.tut.by/film/kolskaya-sverkhglub...,Кольская сверхглубокая скважина - самый масшта...,[Триллер],2020,132 минуты,[Режиссер: Арсений Сюхин],"[Милена Радулович, Никита Дювбанов, Кирилл Ков...","5,7"
2,Побочный эффект,https://img.afisha.tut.by/static/media/640x300...,https://afisha.tut.by/film/pobochnyy_effekt_94...,Однажды в размеренную и счастливую жизнь архит...,"[Триллер, Ужасы]",2020,100 минут,None,"[Милена Радулович, Никита Дювбанов, Кирилл Ков...","1,5"
3,Побочный эффект,https://img.afisha.tut.by/static/media/176x0ec...,https://afisha.tut.by/film/pobochnyy_effekt_94...,Однажды в размеренную и счастливую жизнь архит...,"[Триллер, Ужасы]",2020,100 минут,None,"[Милена Радулович, Никита Дювбанов, Кирилл Ков...","1,5"
4,Кот под прикрытием,https://img.afisha.tut.by/static/media/176x0ec...,https://afisha.tut.by/film/kot_pod_prikrytiem/,Два полицейских спецагента должны раскрыть дел...,[Приключения],2019,90 минут,None,"[Милена Радулович, Никита Дювбанов, Кирилл Ков...","7,8"
...,...,...,...,...,...,...,...,...,...,...
57,Ночь пожирателей рекламы,https://img.afisha.tut.by/static/media/176x0ec...,https://afisha.tut.by/film/noch_pozhirateley_r...,"Уникальное шоу, покорившее весь мир, возвращае...","[Военный, Документальный]",2020,360 минут,None,"[Шарлотта Генсбур, Беатрис Даль, Мика Арганьяр...","10,0"
58,Ночь в Лувре: Леонардо да Винчи,https://img.afisha.tut.by/static/media/176x0ec...,https://afisha.tut.by/film/noch_v_luvre_leonar...,Эксклюзивная ночная экскурсия по выставке «Лео...,[Документальный],2020,100 минут,None,"[Шарлотта Генсбур, Беатрис Даль, Мика Арганьяр...","10,0"
59,TheatreHD: Настоящая комедия (RU SUB),https://img.afisha.tut.by/static/media/176x0ec...,https://afisha.tut.by/film/theatrehd_nastoyash...,Язык: английский,[Документальный],2019,160 минут,None,"[Шарлотта Генсбур, Беатрис Даль, Мика Арганьяр...","7,5"
60,Нулевой канал (RU SUB),https://img.afisha.tut.by/static/media/176x0ec...,https://afisha.tut.by/film/nulevoy_kanal_zero_...,Фильм демонстрируется на языке оригинала с рус...,"[Приключения, Спорт]",2020,130 минут,None,"[Шарлотта Генсбур, Беатрис Даль, Мика Арганьяр...","7,5"
